# Text classification with TensorFlow Hub: Movie reviews

In [3]:
import numpy as np
import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


Version:  2.1.0
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


ERROR: tfds-nightly 3.1.0.dev202004300105 requires future, which is not installed.
ERROR: tfds-nightly 3.1.0.dev202004300105 requires promise, which is not installed.
ERROR: tfds-nightly 3.1.0.dev202004300105 requires tensorflow-metadata, which is not installed.


## Download the IMDB dataset

In [4]:
# IMDB dataset을 다운로드 후 train, test set을 split
# train: test를 60:40으로 split

train_data, validation_data, test_data=tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
    )

Shuffling and writing examples to C:\Users\anna4\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteWMDCXA\imdb_reviews-train.tfrecord


Shuffling and writing examples to C:\Users\anna4\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteWMDCXA\imdb_reviews-test.tfrecord


Shuffling and writing examples to C:\Users\anna4\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteWMDCXA\imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to C:\Users\anna4\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


## Explore the data

In [6]:
# 데이터 1개는 영화 리뷰 1개, label로 이루어짐
# label=0이면 negative, 1이면 positive review

train_examples_batch, train_labels_batch=next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'This is a big step down after the surprisingly enjoyable original. This sequel isn\'t nearly as fun as part one, and it instead spends too much time on plot development. Tim Thomerson is still the best thing about this series, but his wisecracking is toned down in this entry. The performances are all adequate, but this time the script lets us down. The action is merely routine and the plot is only mildly interesting, so I need lots of silly laughs in order to stay entertained during a "Trancers" movie. Unfortunately, the laughs are few and far between, and so, this film is watchable at best.',
       b"Perhaps because I was so young, innocent and BRAINWASHED when I saw it, this movie was the cause of many sleepless nights for me. I haven't seen it since I was in seventh grade at a Presbyterian school, so I am not sure what effect it would have on me now. However, I will say that it left an impression on me... and most of my friends

In [7]:
# label 10개 출력해보기

train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 1, 0, 1, 0, 1, 1, 1, 0], dtype=int64)>

## Build the model

- text를 표현하는 방법: sentence를 embedding vector로 변환
- embedding vector의 장점: preprocessing 신경X, transfer learning 가능, 크기가 고정되어있음
- 여기서는 TensorFlow Hub에서 미리 학습된 text embedding model인  google/tf2-preview/gnews-swivel-20dim/1을 사용할 것임

In [10]:
# 출력 크기는 (num_exmamples, embedding_dimension)

embedding="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer=hub.KerasLayer(embedding, input_shape=[],
                        dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 2.209591  , -2.7093675 ,  3.6802928 , -1.0291991 , -4.1671185 ,
        -2.4566064 , -2.2519937 , -0.36589956,  1.9485804 , -3.1104462 ,
        -2.4610963 ,  1.3139242 , -0.9161584 , -0.16625322, -3.723651  ,
         1.8498232 ,  3.499562  , -1.2373022 , -2.8403084 , -1.213074  ],
       [ 1.9055302 , -4.11395   ,  3.6038654 ,  0.28555924, -4.658998  ,
        -5.5433393 , -3.2735848 ,  1.9235417 ,  3.8461034 ,  1.5882455 ,
        -2.64167   ,  0.76057523, -0.14820506,  0.9115291 , -6.45758   ,
         2.3990374 ,  5.0985413 , -3.2776263 , -3.2652326 , -1.2345369 ],
       [ 3.6510668 , -4.7066135 ,  4.71003   , -1.7002777 , -3.7708545 ,
        -3.709126  , -4.222776  ,  1.946586  ,  6.1182513 , -2.7392752 ,
        -5.4384456 ,  2.7078724 , -2.1263676 , -0.7084146 , -5.893995  ,
         3.1602864 ,  3.8389287 , -3.318196  , -5.1542974 , -2.4051712 ]],
      dtype=float32)>

In [13]:
# 전체 모델

model=tf.keras.Sequential()
model.add(hub_layer)  # 텐서플로 허브 층
# 사전훈련된 모델을 사용해 하나의 문장을 임베딩 벡터로 매핑함?
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


## Loss function and optimizer

In [14]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])

## Train the model

In [16]:
history=model.fit(train_data.shuffle(10000).batch(512),
                 epochs=20,
                 validation_data=validation_data.batch(512),
                 verbose=1)

Epoch 1/20
30/30 [==============================] - 5s 155ms/step - loss: 0.6893 - accuracy: 0.5443 - val_loss: 0.6773 - val_accuracy: 0.5256
Epoch 2/20
30/30 [==============================] - 4s 119ms/step - loss: 0.6676 - accuracy: 0.5884 - val_loss: 0.6592 - val_accuracy: 0.6351
Epoch 3/20
30/30 [==============================] - 4s 117ms/step - loss: 0.6537 - accuracy: 0.6573 - val_loss: 0.6493 - val_accuracy: 0.6524
Epoch 4/20
30/30 [==============================] - 3s 114ms/step - loss: 0.6439 - accuracy: 0.6761 - val_loss: 0.6416 - val_accuracy: 0.6775
Epoch 5/20
30/30 [==============================] - 4s 120ms/step - loss: 0.6354 - accuracy: 0.7028 - val_loss: 0.6340 - val_accuracy: 0.6996
Epoch 6/20
30/30 [==============================] - 4s 123ms/step - loss: 0.6267 - accuracy: 0.7264 - val_loss: 0.6265 - val_accuracy: 0.7150
Epoch 7/20
30/30 [==============================] - 4s 121ms/step - loss: 0.6178 - accuracy: 0.7546 - val_loss: 0.6186 - val_accuracy: 0.7438
Epoch 

## Evaluate the model

In [17]:
# loss는 낮을수록, accuracy는 높을수록 좋음
results=model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

loss: 0.575
accuracy: 0.854
